<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#!/usr/bin/env python3
# ================================================================
# MY-AI-Gizmo • LAUNCHER — FIXED
# - Prevents long silent "Finishing up" hangs by skipping llama.cpp build
# - Renames llama.cpp repo to avoid automatic rebuild (safe fallback)
# - Streams installer output live with heartbeat so it never appears stuck
# - Forces matplotlib backend inside server process to Agg to avoid backend errors
# - Captures Gradio public URL and writes it to Drive
# - Persists logs, settings, models, chat-history to DRIVE_ROOT
# Intended for Google Colab / Linux with Drive mounted at /content/drive
# ================================================================

import os
import subprocess
import shutil
import re
import time
import threading
from pathlib import Path

try:
    from google.colab import drive as colab_drive
    IN_COLAB = True
except Exception:
    colab_drive = None
    IN_COLAB = False

# ---------- Configuration ----------
REPO_ZIP = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR = Path("/content/text-generation-webui")
DRIVE_ROOT = Path("/content/drive/MyDrive/MY-AI-Gizmo")
LOG_DIR = DRIVE_ROOT / "logs"
MPL_CONFIG_DIR = DRIVE_ROOT / "matplotlib"
HEARTBEAT_INTERVAL = 30  # seconds
PUBLIC_URL_FILE = DRIVE_ROOT / "public_url.txt"
# -----------------------------------

def sh(cmd, cwd=None, env=None, check=False):
    return subprocess.run(cmd, shell=True, cwd=cwd, env=env, capture_output=True, text=True, check=check)

def stream_with_heartbeat(cmd, cwd=None, env=None, logfile_path=None, capture_url_to=None):
    """
    Run command, stream output line-by-line, print heartbeat if silent.
    Capture first Gradio/public URL found and optionally save it to capture_url_to.
    Returns (returncode, captured_url_or_None).
    """
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                            cwd=cwd, env=env, text=True, bufsize=1)

    last_output = time.time()
    stop = threading.Event()
    captured_url = None

    url_patterns = [
        re.compile(r"(https?://[^\s)'\"]+gradio\.\w+[^\s)'\"]*)", re.IGNORECASE),
        re.compile(r"(https?://[^\s)'\"]+\.gradio\.live[^\s)'\"]*)", re.IGNORECASE),
        re.compile(r"(https?://[^\s)'\"]+:\d+)", re.IGNORECASE),
        re.compile(r"(https?://[^\s)'\"]+)", re.IGNORECASE),
    ]

    def heartbeat():
        while not stop.wait(HEARTBEAT_INTERVAL):
            if time.time() - last_output >= HEARTBEAT_INTERVAL:
                msg = f"[heartbeat] still working... (no output for ~{HEARTBEAT_INTERVAL}s)\n"
                print(msg, end='')
                if logfile_path:
                    try:
                        with open(logfile_path, "a", encoding="utf-8") as f:
                            f.write(msg)
                    except Exception:
                        pass

    hb = threading.Thread(target=heartbeat, daemon=True)
    hb.start()

    logfile = None
    if logfile_path:
        try:
            logfile = open(logfile_path, "a", encoding="utf-8")
        except Exception:
            logfile = None

    try:
        for line in proc.stdout:
            last_output = time.time()
            print(line, end='')
            if logfile:
                try:
                    logfile.write(line)
                except Exception:
                    pass
            if not captured_url:
                for pat in url_patterns:
                    m = pat.search(line)
                    if m:
                        candidate = m.group(1).rstrip(').,\'"')
                        # prefer gradio hosts
                        if 'gradio' in candidate.lower() or ':' in candidate:
                            captured_url = candidate
                            break
                        elif not captured_url:
                            captured_url = candidate
                if captured_url and capture_url_to:
                    try:
                        Path(capture_url_to).write_text(captured_url, encoding="utf-8")
                    except Exception:
                        pass
    except Exception as e:
        print(f"[stream error] {e}")
    finally:
        proc.wait()
        stop.set()
        hb.join(timeout=1)
        if logfile:
            try:
                logfile.close()
            except Exception:
                pass

    return proc.returncode, captured_url

def ensure_dirs():
    for d in (DRIVE_ROOT, LOG_DIR, MPL_CONFIG_DIR):
        try:
            d.mkdir(parents=True, exist_ok=True)
        except Exception:
            pass

def download_repo_if_missing():
    if WORK_DIR.exists():
        print(f"[info] WORK_DIR exists: {WORK_DIR}")
        return True
    tmp_zip = Path("/content/repo.zip")
    try:
        tmp_zip.unlink()
    except Exception:
        pass
    print("[info] downloading repository...")
    ok = False
    for cmd in (f"wget -q -O {tmp_zip} {REPO_ZIP}", f"curl -s -L -o {tmp_zip} {REPO_ZIP}"):
        try:
            sh(cmd)
        except Exception:
            pass
        if tmp_zip.exists() and tmp_zip.stat().st_size > 1000:
            ok = True
            break
    if not ok:
        print("[error] download failed. Check network/URL.")
        return False
    print("[info] extracting...")
    try:
        sh(f"unzip -q {tmp_zip} -d /content")
        found = next(Path("/content").glob("MY-AI-Gizmo-working-*"), None)
        if not found:
            print("[error] expected extracted folder not found")
            return False
        found.rename(WORK_DIR)
        print("[info] repo extracted to", WORK_DIR)
        return True
    except Exception as e:
        print("[error] extract failed:", e)
        return False

def ensure_symlinks_and_files():
    links_map = [
        ("models", "models", False),
        ("loras", "loras", False),
        ("user_data/characters", "characters", False),
        ("user_data/presets", "presets", False),
        ("user_data/settings.yaml", "settings/settings.yaml", True),
        ("user_data/settings.json", "settings/settings.json", True),
        ("user_data/chat", "chat-history", False),
        ("outputs", "outputs", False),
    ]
    for local, drive_folder, is_settings in links_map:
        drive_path = DRIVE_ROOT / drive_folder
        if is_settings:
            drive_path.parent.mkdir(parents=True, exist_ok=True)
            if not drive_path.exists():
                try:
                    drive_path.write_text("", encoding="utf-8")
                except Exception:
                    pass
        else:
            drive_path.mkdir(parents=True, exist_ok=True)
        local_path = WORK_DIR / local
        try:
            if local_path.exists() or local_path.is_symlink():
                if local_path.is_symlink():
                    local_path.unlink()
                elif local_path.is_dir():
                    shutil.rmtree(local_path)
                else:
                    local_path.unlink()
        except Exception:
            pass
        # attempt symlink; fallback to copy if symlink not allowed
        try:
            local_path.parent.mkdir(parents=True, exist_ok=True)
            os.symlink(str(drive_path), str(local_path), target_is_directory=drive_path.is_dir())
        except Exception:
            try:
                if drive_path.is_dir():
                    shutil.copytree(drive_path, local_path)
                else:
                    local_path.parent.mkdir(parents=True, exist_ok=True)
                    shutil.copy2(drive_path, local_path)
            except Exception:
                pass

def prepare_settings_file():
    drive_settings = DRIVE_ROOT / "settings" / "settings.yaml"
    local_settings = WORK_DIR / "user_data" / "settings.yaml"
    local_settings.parent.mkdir(parents=True, exist_ok=True)
    try:
        if drive_settings.exists() and drive_settings.stat().st_size > 0:
            shutil.copy2(drive_settings, local_settings)
        else:
            if not local_settings.exists() or local_settings.stat().st_size == 0:
                local_settings.write_text("# minimal\nlisten: true\nshare: true\n", encoding="utf-8")
                drive_settings.parent.mkdir(parents=True, exist_ok=True)
                try:
                    shutil.copy2(local_settings, drive_settings)
                except Exception:
                    pass
    except Exception:
        pass

# ---------- Main flow ----------
ensure_dirs()

if IN_COLAB:
    try:
        colab_drive.mount("/content/drive", force_remount=False)
    except Exception:
        pass

# cleanup broken files in Drive models folder (small incomplete files)
def cleanup_broken_files(drive_root: Path):
    models_dir = drive_root / "models"
    if not models_dir.exists():
        return
    extensions = ["*.gguf", "*.safetensors", "*.bin", "*.pth", "*.pt"]
    broken = []
    for ext in extensions:
        for f in models_dir.rglob(ext):
            try:
                if f.stat().st_size < (100 * 1024):
                    broken.append(f)
            except Exception:
                pass
    if broken:
        for f in broken:
            try:
                f.unlink()
            except Exception:
                pass

cleanup_broken_files(DRIVE_ROOT)

# repo
if not download_repo_if_missing() and not WORK_DIR.exists():
    raise SystemExit("Repository unavailable. Fix network/REPO_ZIP and retry.")

os.chdir(WORK_DIR)

ensure_symlinks_and_files()
prepare_settings_file()

# Prepare to avoid long llama.cpp compile: rename folder if present (safe)
llama_dir = WORK_DIR / "repositories" / "llama.cpp"
llama_disabled = None
try:
    if llama_dir.exists() and llama_dir.is_dir():
        llama_disabled = llama_dir.with_name(llama_dir.name + ".disabled")
        if llama_disabled.exists():
            shutil.rmtree(llama_disabled, ignore_errors=True)
        llama_dir.rename(llama_disabled)
        print("[info] renamed repositories/llama.cpp ->", llama_disabled.name, "(prevents auto-build)")
except Exception:
    llama_disabled = None

# Installer (Step 6) — run start_linux.sh with fast flags and live streaming
start_sh = WORK_DIR / "start_linux.sh"
installer_log = LOG_DIR / f"installer_{int(time.time())}.log"
env_marker = WORK_DIR / "installer_files" / "env" / "bin" / "python"
install_env = os.environ.copy()
install_env["MPLBACKEND"] = "Agg"
install_env["MPLCONFIGDIR"] = str(MPL_CONFIG_DIR)
install_env.update({
    "GPU_CHOICE": "A",
    "LAUNCH_AFTER_INSTALL": "FALSE",   # do not auto-launch after install
    "INSTALL_EXTENSIONS": "FALSE",
    "SKIP_LLAMACPP_BUILD": "TRUE",     # best-effort skip of llama.cpp compile
    "SKIP_TORCH_TEST": "TRUE",
    "FORCE_CUDA": "FALSE",
})

print("\n📦 Step 6/6: Installing dependencies (fast mode)...")
print("Installer log ->", installer_log)
sh("chmod +x start_linux.sh")

if env_marker.exists():
    print("[info] virtualenv exists; skipping full install")
else:
    if start_sh.exists():
        code, url = stream_with_heartbeat("bash start_linux.sh", cwd=str(WORK_DIR), env=install_env, logfile_path=str(installer_log), capture_url_to=str(PUBLIC_URL_FILE))
        if code != 0:
            print(f"[warn] installer exited with code {code}. See {installer_log} for details.")
        else:
            print(f"[info] installer finished successfully (log: {installer_log})")
    else:
        print("[warn] start_linux.sh not found; install skipped. You may need to run installation steps manually.")

# restore llama.cpp folder name so user can build manually later (it won't be auto-built unless script forces it)
if llama_disabled:
    try:
        restored = WORK_DIR / "repositories" / "llama.cpp"
        if not restored.exists():
            llama_disabled.rename(restored)
            print("[info] restored repositories/llama.cpp (manual build possible later)")
    except Exception:
        pass

# Ensure MPLCONFIGDIR exists and is writable, and force MPLBACKEND in launcher env
MPL_CONFIG_DIR.mkdir(parents=True, exist_ok=True)

# LAUNCH: create small wrapper that forces matplotlib backend INSIDE server process
wrapper = WORK_DIR / "_run_server_with_agg.py"
wrapper_code = f"""# autogenerated wrapper: enforce MPL backend inside server process
import os, runpy
os.environ['MPLBACKEND'] = 'Agg'
os.environ['MPLCONFIGDIR'] = r'{MPL_CONFIG_DIR}'
try:
    import matplotlib
    matplotlib.use('Agg', force=True)
except Exception:
    pass
runpy.run_path('server.py', run_name='__main__')
"""
try:
    wrapper.write_text(wrapper_code, encoding="utf-8")
    wrapper.chmod(0o755)
except Exception:
    pass

# Kill stray python processes that might block ports
try:
    sh("pkill -9 -f python")
except Exception:
    pass
time.sleep(1.5)

server_log = LOG_DIR / f"server_{int(time.time())}.log"
python_exe = str(env_marker) if env_marker.exists() else "python3"
launch_cmd = f'{python_exe} -u "{wrapper.name}"'
print("\n🌐 LAUNCHING UI")
print("Server log ->", server_log)
code, captured = stream_with_heartbeat(launch_cmd, cwd=str(WORK_DIR), env=dict(os.environ, MPLBACKEND="Agg", MPLCONFIGDIR=str(MPL_CONFIG_DIR)), logfile_path=str(server_log), capture_url_to=str(PUBLIC_URL_FILE))

if captured:
    print("\n" + "=" * 70)
    print("[success] Public or local URL detected:", captured)
    try:
        PUBLIC_URL_FILE.write_text(captured, encoding="utf-8")
        print("[info] URL saved to:", PUBLIC_URL_FILE)
    except Exception:
        pass
else:
    print("[warn] No URL captured from server output. Check server log:", server_log)

if code != 0:
    print(f"[warn] server exited with code {code}. See {server_log}")
else:
    print(f"[info] server terminated normally. See {server_log}")

print("\n✓ Done. Persistent data, logs and settings are in:", DRIVE_ROOT)


✅ RECOMMENDED MODELS (COPY EXACTLY)
🔹 BEST GENERAL CHAT (START HERE)

Llama-2-7B-Chat

Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf

🔹 FAST + LIGHT (LOW RAM)

TinyLlama-1.1B-Chat

Repo: TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF
File: tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

🔹 STRONG CHAT (BETTER THAN LLAMA-2)

Mistral-7B-Instruct

Repo: TheBloke/Mistral-7B-Instruct-v0.2-GGUF
File: mistral-7b-instruct-v0.2.Q4_K_M.gguf

🔹 CODING MODEL

Code LLaMA-7B

Repo: TheBloke/CodeLlama-7B-GGUF
File: codellama-7b.Q4_K_M.gguf

🔹 ROLEPLAY / STORY

MythoMax-L2-13B (needs more RAM)

Repo: TheBloke/MythoMax-L2-13B-GGUF
File: mythomax-l2-13b.Q4_K_M.gguf

🔹 VERY FAST / TEST MODEL

Phi-2 (2.7B)

Repo: TheBloke/phi-2-GGUF
File: phi-2.Q4_K_M.gguf

⚙️ WHAT LOADER TO USE (IMPORTANT)

For ALL models above:

Loader: llama.cpp


Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf
